В этом задании надо разобраться с тем, какие признаки есть в датасете, как их надо готовить для конкретных моделей (пропуски, выбросы, различные преобразования). 
Результат работы надо сохранить - на семинаре будет активность, для которой блоки фичей, подготовленные для разных моделей, будут очень полезны. 

In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

In [4]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

### Признаков много, надо их разделить на группы

In [5]:
train.dtypes

name                                 object
summary                              object
space                                object
description                          object
experiences_offered                  object
neighborhood_overview                object
notes                                object
transit                              object
access                               object
interaction                          object
house_rules                          object
host_id                               int64
host_since                           object
host_about                           object
host_response_time                   object
host_response_rate                   object
host_is_superhost                    object
host_has_profile_pic                 object
host_identity_verified               object
neighbourhood_cleansed               object
zipcode                              object
latitude                            float64
longitude                       

In [6]:
# Сначала выделим текст и даты. dtypes нам на них не указал - придется находить их глазами

In [7]:
train.head(3)

,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,host_id,host_since,host_about,host_response_time,host_response_rate,host_is_superhost,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,zipcode,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,price
0,Double bedroom in cottage Twickenham (sleeps 1-2),"One gorgeous, light-filled double bedroom (sle...","A beautiful, light-filled double bedroom is ou...","One gorgeous, light-filled double bedroom (sle...",none,"Very, very safe area, great transport links an...",We have a super-gorgeous cat,Twickenham Rugby Stadium is 15 mins walk Water...,"You'll have full use of the cottage, two recep...","The host (Andy) is usually in residence, but m...","While the house is a great space to entertain,...",161720,2010-07-09,My wife and I work in property development. S...,NaN,NaN,f,t,t,Richmond upon Thames,TW2 6,51.447147,-0.339253,t,House,Private room,2,1.0,1.0,1.0,Real Bed,"{TV,""Cable TV"",Internet,Wifi,Kitchen,Breakfast...",NaN,300.0,10.0,1,10.0,2,strict_14_with_grace_period,f,f,1000.0
1,Big House for Olympics sleeps 6 to8,NaN,"A beautiful, modern, art-filled and clean 4 st...","A beautiful, modern, art-filled and clean 4 st...",none,NaN,NaN,NaN,NaN,NaN,A copy of your passports or Id cards will be r...,1666422,2012-01-28,"Hi there,\r\n\r\nI'm a music artist manager an...",NaN,NaN,f,t,f,Camden,NW3 6NE,51.549993,-0.184546,t,House,Entire home/apt,8,2.5,3.0,4.0,Real Bed,"{""Cable TV"",Internet,Wifi,Kitchen,""Free parkin...",1400.0,771.0,NaN,1,0.0,14,strict_14_with_grace_period,f,f,771.0
2,The Old Coach House (Olympics),NaN,"Fabulous, recently refurbished original Victor...","Fabulous, recently refurbished original Victor...",none,NaN,NaN,NaN,NaN,NaN,NaN,2132532,2012-04-13,"I am making available a fabulous, recently ref...",NaN,NaN,f,t,f,Lewisham,SE10,51.469213,-0.007671,t,House,Entire home/apt,6,3.0,3.0,3.0,Real Bed,"{TV,Internet,Wifi,Kitchen,""Free parking on pre...",1800.0,600.0,NaN,1,0.0,14,strict_14_with_grace_period,f,f,1500.0


In [8]:
text_cols = ['name', 'summary', 'space', 'description', 'experiences_offered',
             'neighborhood_overview', 'notes', 'transit', 'access', 'interaction', 'house_rules', 'host_about']

date_cols = ['host_since']

In [9]:
# Выделим числовые признаки

In [10]:
train.select_dtypes(include=['float64', 'int64'])

num_cols = train.select_dtypes(include=['float64', 'int64']).index
train[num_cols].head(3)

,host_id,latitude,longitude,accommodates,bathrooms,bedrooms,beds,square_feet,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,price
0,161720,51.447147,-0.339253,2,1.0,1.0,1.0,NaN,300.0,10.0,1,10.0,2,1000.0
1,1666422,51.549993,-0.184546,8,2.5,3.0,4.0,1400.0,771.0,NaN,1,0.0,14,771.0
2,2132532,51.469213,-0.007671,6,3.0,3.0,3.0,1800.0,600.0,NaN,1,0.0,14,1500.0


In [11]:
# Числовыми оказались не все признаки:
num_cols = num_cols.drop(['host_id', 'latitude', 'longitude'])
geo_cols = ['latitude', 'longitude']
# host_id - категориальный признак

In [ ]:
# Выделим категориальные признаки

In [5]:
cat_cols = np.setdiff1d(train.select_dtypes(include=['object']).index, text_cols+date_cols)
train[cat_cols].head(3)

,amenities,bed_type,cancellation_policy,host_has_profile_pic,host_identity_verified,host_is_superhost,host_response_rate,host_response_time,is_location_exact,neighbourhood_cleansed,property_type,require_guest_phone_verification,require_guest_profile_picture,room_type,zipcode
0,"{TV,""Cable TV"",Internet,Wifi,Kitchen,Breakfast...",Real Bed,strict_14_with_grace_period,t,t,f,NaN,NaN,t,Richmond upon Thames,House,f,f,Private room,TW2 6
1,"{""Cable TV"",Internet,Wifi,Kitchen,""Free parkin...",Real Bed,strict_14_with_grace_period,t,f,f,NaN,NaN,t,Camden,House,f,f,Entire home/apt,NW3 6NE
2,"{TV,Internet,Wifi,Kitchen,""Free parking on pre...",Real Bed,strict_14_with_grace_period,t,f,f,NaN,NaN,t,Lewisham,House,f,f,Entire home/apt,SE10


In [10]:
# Все-таки host_response_rate - числовой признак: 
train.host_response_rate.value_counts().shape

(74,)

In [6]:
train['host_response_rate'] = train['host_response_rate'].str[:-1].astype(float)
test['host_response_rate'] = test['host_response_rate'].str[:-1].astype(float)
num_cols = num_cols.tolist() + ['host_response_rate']

amenities - признак который нельзя просто засунуть в модель, подумайте, что с ним делать

In [7]:
cat_cols = cat_cols.tolist() + ['host_id']
cat_cols.remove('amenities')
cat_cols.remove('host_response_rate')

In [ ]:
# Что еще важно знать про категориальные признаки

In [8]:
train.drop('price', axis=1).append(test)[cat_cols].nunique()

bed_type                                5
cancellation_policy                     6
host_has_profile_pic                    2
host_identity_verified                  2
host_is_superhost                       2
host_response_time                      4
is_location_exact                       2
neighbourhood_cleansed                 33
property_type                          42
require_guest_phone_verification        2
require_guest_profile_picture           2
room_type                               3
zipcode                             31278
host_id                             48941
dtype: int64

Категориальные признаки удобно разделять по мощности - вряд ли вы захотите сделать one hot encoding для признака zipcode

In [9]:
# Заполнять пропуски для категориальных признаков сплошное удовольствие, сделаем это сразу
train[cat_cols] = train[cat_cols].fillna('other')
test[cat_cols] = test[cat_cols].fillna('other')

In [ ]:
# В eda мы находили порядковые признаки

In [10]:
train.room_type.value_counts(dropna=False)

Entire home/apt    29567
Private room       21732
Shared room          516
Name: room_type, dtype: int64

In [22]:
test.room_type.value_counts(dropna=False)

Entire home/apt    11568
Private room       11232
Shared room          200
Name: room_type, dtype: int64

В pandas есть возможность указать, что признак является категориальным (это существенно экономит память)
Порядок в котором кодируются категории задается вот так:

In [25]:
train.room_type = train.room_type.astype('category').cat.set_categories(['Shared room', 
                                                                         'Private room', 'Entire home/apt'], ordered=True)
test.room_type = test.room_type.astype('category').cat.set_categories(['Shared room', 
                                                                       'Private room', 'Entire home/apt'], ordered=True)

In [28]:
train.room_type.cat.categories

Index(['Shared room', 'Private room', 'Entire home/apt'], dtype='object')

In [30]:
# Так можно перейти к кодам, не используя label encoding

train.room_type.cat.codes[:5]

0    1
1    2
2    2
3    2
4    2
dtype: int8

 Найдите еще какие-нибудь порядковые признаки и задайте им порядок

### Обработайте числовые признаки. Подумайте как бы вы заполнили пропуски, если бы готовили признаки для линейной модели,  для деревьев итд. 
Ищите вдохновение в классе sklearn.preprocessing (StandardScaler, PolynomialFeatures... - вот это вот все). Блоки фичей (для деревьев, для линейных моделей) сохраните.

In [ ]:
# ваш код

### Обработайте категориальные признаки. Попробуйте разные способы: OneHotEncoder, LabelEncoder, проекции числовых признаков на категориальные, кодирование частотой или чем-нибудь еще
Блоки фичей (для деревьев, для линейных моделей) сохраните. Заранее подумайте о том, как дальше работать с этими блоками, как вы будете их собирать в датасет, что будет, если вы захотите посмотреть на важность признаков и понять, что это за фича номер 8 в топе.

In [ ]:
# ваш код

### Даты модель обрабатывать не умеет, сделайте это для нее. 
Можно кодировать даты количеством времени до-после какого-то события, или извлекать признаки, связанные календарной с датой: день недели, месяц время года, декада луны. 
Не забудьте сохранить

In [ ]:
# ваш код

### И наконец обработайте тексты: можно попробовать все вот отсюда sklearn.feature_extraction.text, можно разобраться с эмбеддингами (взять из fastText). 
Сохраните

In [ ]:
# Ваш код